In [ ]:
try:
    all_features = pd.from_csv('all_features.csv')
except FileNotFoundError:
    all_dt = RvBsm['Device & Trip'].unique()
    all_features = pd.from_csv('features/'+all_dt.iloc[0]+'.csv')
    for dt in all_dt[1:]:
        all_features = all_features.append(pd.from_csv('features/'+dt+'.csv'))
    all_features.to_csv('all_features.csv')
all_features.head()

In [ ]:
try:
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
except FileNotFoundError:
    split_index = (len(all_features) * 0.8) //1

    train_y = all_features['id'].iloc[:split_index]
    test_y = all_features['id'].iloc[split_index:]
    train_X = all_features.iloc[split_index:].drop( 'id', axis = 1 )
    test_X = all_features.iloc[:split_index].drop( 'id', axis = 1 )

    train_features_selected = select_features( train_x, train_y, fdr_level = 0.05 )

    train = train_features_selected.copy()
    train['id'] = train_y

    test = test_x[ train_features_selected.columns ].copy()
    test['id'] = test_y

    train.to_csv('train.csv', index = None )
    test.to_csv('test.csv', index = None )

In [ ]:
train = pd.read_csv( train_file )
test = pd.read_csv( test_file )

train_X = train.drop('id', axis = 1 ).values
test_X = test.drop('id', axis = 1 ).values

train_y = train['id'].values
test_y = test['id'].values

classifiers = [make_pipeline( StandardScaler(), LR()),
    make_pipeline( MinMaxScaler(), LR()),
    RF(n_estimators = 100, min_samples_leaf = 5)]

for clf in classifiers:
    clf.fit(train_X, train_y)
    p = clf.predict_proba(test_X)[:,1]
    p_bin = clf.predict(test_X)

    auc = AUC(test_y, p )
    acc = accuracy(test_y, p_bin )